# Using states' Secretary of State data to learn about organizations


## Outline:
    1. Scrape some data and load it into Elasticsearch
    2. Use Kibana somehow?
    3. Query Elasticsearch for data to put into a pandas dataframe
    4. Prep the dataframe by adding new variables
    5. Make models and learn something new about companies in Rhode Island

### Before starting, make sure you have ready:
    - Elasticsearch (installed and running) (both the program & the Python package)
    - Tor (installed and running)
    - bs4, aka BeautifulSoup
    - pandas
    - pysocks isn't imported, but it must be installed

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import requests
import json
import random
import re
import pickle
import math
import collections
import elasticsearch.helpers
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from elasticsearch import Elasticsearch
from collections import Counter
from sklearn.model_selection import train_test_split

# Step 1: scrape some data

### Scraping data can be illegal! I'm not recommending you do it, just showing you how easy it is to do. If you're interested in the analysis, you can buy this full database from the Rhode Island Secretary of State for fifty bucks.


### We're using Tor just to show how to use Tor. Unless you're scraping the dark web, using Tor is pretty impractical. Tor is orders of magnitude slower than normal requests.

### The data we collect is being stored in Elasticsearch. You could keep it around in memory, but it takes a long time to scrape data, so it's better to save it to a database so you can quickly use it again in the future if need be.

In [3]:
def get_tor_session():
    session = requests.session()
    # Set up your proxies to use Tor
    session.proxies = {'http': 'socks5://localhost:9150',
                       'https': 'socks5://localhost:9150'}
    return session

In [4]:
def try_and_except_loop(soup, data_dict, value, prefix, value_not_text=False):
    """A short loop to keep things a little DRYer"""
    if value_not_text:
        try:
            data_dict[value] = soup.find(id=prefix + str(value)).value
        except AttributeError:
            pass
    else:
        try:
            data_dict[value] = soup.find(id=prefix + str(value)).text
        except AttributeError:
            pass

In [5]:
# After inspecting the web page we're going to be grabbing, I got the names of the worthwhile fields:
VALUES_TO_EXTRACT = ["EntityName",
                    "EntityType",
                    "OrganisationDate", # Yes, this is misspelled in the HTML
                    "EffectiveDate",
                    "InactiveDate",
                    "PrincipleStreet",
                    "PrincipleCity",
                    "PrincipleState",
                    "PrincipleZip",
                    "PrincipleCountry",
                    "ResidentAgentFlag",
                    "ConsentFlag",
                    "ResidentAgentName",
                    "ResidentStreet",
                    "ResidentCity",
                    "ResidentState",
                    "ResidentZip",
                    "ResidentCountry"]

def parse_rhode_island_entity_summary(session):
    """When given Rhode Island's Entity Summary page, this builds a dictionary of
        all the noteworthy data on the webpage"""
    data_dict = {}
    soup = BeautifulSoup(session._content)

    # This value includes the words "Identification Number: ". Clear that useless text.
    # Also, if there is no ID number present, then there's no other information present either,
    # and there's no need to continue selecting data.
    try:
        data_dict["IDNumber"] = soup.find(id="MainContent_lblIDNumber").text[-9:]
    except AttributeError:
        return False

    for item in VALUES_TO_EXTRACT:
        try_and_except_loop(soup, data_dict, item, "MainContent_lbl")
    try_and_except_loop(soup, data_dict, "Comments", "MainContent_txt")
    try_and_except_loop(soup, data_dict, "NIACS", "MainContent_txt", value_not_text=True)

    # The length of the list of directors & officers varies by company
    d_and_o_list = []
    table = soup.find(id="MainContent_grdOfficers")
    try:
        for row in table.find_all('tr')[1:]:
            col = row.find_all('td')
            d_and_o_list += [col[0].text, col[1].text, col[2].text]
    except AttributeError:
        # Some pages don't have information on directors & officers
        pass
    data_dict['d_and_o'] = d_and_o_list

    # I have no idea if their data on stocks is meaningful, but just in case:
    try:
        stock_cols = soup.find(id="MainContent_grdStocks").tbody.tr.find_all('td')
        data_dict['stock_info'] = [stock_cols[item].text for item in range(len(stock_cols))]
    except AttributeError:
        # Some pages don't have information on stocks. Makes sense, since all the info isn't on corporations.
        pass

    return data_dict

In [6]:
es = Elasticsearch()

def create_elasticsearch_index():
    """This should only be run once: re-running it will delete any saved data!"""
    es.indices.delete(index="ri_sos", ignore=[400, 404])
    es_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "corporate_data": {
                "properties": {
                    "OrganisationDate": {
                        "type": "date",
                        "format": "MM-dd-yyyy"
                    },
                    "EffectiveDate": {
                        "type": "date",
                        "format": "MM-dd-yyyy"
                    },
                    "InactiveDate": {
                        "type": "date",
                        "format": "MM-dd-yyyy"
                    },
                    "IDNumber": {
                        "type": "integer"
                    }
                }
            }
        }}
    es.indices.create(index="ri_sos", body=es_settings)

In [7]:
def save_data_in_elasticsearch(returned_page):
    """Updates elasticsearch with a single row of data"""
    try:
        es.index(index="ri_sos", doc_type="corporate_data", body=json.dumps(returned_page))
    except:
        # Don't let the scraping stop in case some page has an unusual structure.
        # Instead, print out the bad data & figure out the exceptions later.
        print("Didn't save:", returned_page)

In [8]:
def collect_rhode_island_data(start_at=0, records_to_get=1, create_index=False, random_subset_size=False, prior_records=False,
                              failed_records=[]):
    """This function ties together connecting with Tor, scraping data, and loading it into Elasticsearch.
        It returns a list of all records it unsuccessfully tried to grab."""
    URL_string = "http://ucc.state.ri.us/CorpWeb/CorpSearch/CorpSummary.aspx?FEIN="
    session = get_tor_session()
    # Since we're already using Tor, let's pretend we're accessing the web through a browser
    # instead of letting people see it's through a Python script:
    headers = {'User-agent': 'Opera/1.1.2 FCS'}

    ## If you want to confirm that you're using Tor, then see if your
    ## session's IP address matches a normal request's IP address
    # print("TOR IP address:", session.get("http://httpbin.org/ip").text)
    # print("Your normal IP address:", requests.get("http://httpbin.org/ip").text)

    # If this is your first time through, then you'll need to create an index in Elasticsearch
    if create_index:
        create_elasticsearch_index()

    # Options for random sampling of data
    if random_subset_size:
        records = random.sample(range(165000), random_subset_size)
    else:
        records = [start_at + item for item in range(records_to_get)]
    if prior_records:
        records = set(records) - set(prior_records)

    for record in records:
        corp_id = str(start_at + record)
        # The string provided should be 9 digits long, padded by prepended by 0s
        corp_id = corp_id.rjust(9, "0")

        returned_page = session.get(URL_string + str(corp_id), headers=headers)
        data_dict = parse_rhode_island_entity_summary(returned_page)
        if data_dict:
            # Value will be false if the page scraped is blank, so no need to save anything
            save_data_in_elasticsearch(data_dict)
        else:
            failed_records += [record]

        # Clear cookies to make it harder to track you between requests:
        session.cookies.clear()
    return failed_records

## Uncomment & run this if it's your first time running this script:

In [ ]:
# Now we finally start collecting some data
#### ONLY RUN THIS ONCE! OTHERWISE IT WILL DELETE YOUR INDEX, DELETING ALL OF YOUR DATA!
# failed_records = collect_rhode_island_data(start_at=0, 
#                                           records_to_get=5,
#                                           create_index=True)

## Otherwise load your previously saved list of failed records:

In [ ]:
# Load your list of pickled failed records:

#with open('failed_records', 'rb') as fp:
#    failed_records = pickle.load(fp)

### Rhode Island has about 165,000 companies that follow the expected URL pattern. At 1 request/second, that's ~46 hours of scraping to get everything. We could speed up the process by making bulk entries into Elasticsearch rather than making individual entries, by making multiple requests at a time, or using normal requests instead of Tor, but we don't want to overwhelm the server. Always be nice to whoever has to maintain the servers you're talking to. Scraping slowly is nice.

### We don't need the full database to do some analysis. Instead of scraping everything, let's randomly pull some samples and study that subset.

In [ ]:
failed_records = collect_rhode_island_data(start_at=0,
                                           random_subset_size=5,
                                           failed_records=failed_records)

In [11]:
# Pull prior_records from Elasticsearch so you don't pull them again:
def get_prior_records(failed_records):
    """Return a list of all the IDNumbers saved in Elasticsearch, including any failed_records you want to exclude"""
    records_search = elasticsearch.helpers.scan(
        es, 
        query={"query": {"match_all": {}}, "_source":["IDNumber"]},
        index="ri_sos",
        doc_type="corporate_data")

    # Return these records, plus the failed_records we already knew about
    return [item['_source']['IDNumber'] for item in list(records_search)] + failed_records

In [12]:
# Scrape some new records, but don't repeat anything that's already saved in Elasticsearch:
failed_records = collect_rhode_island_data(start_at=0,
                                           random_subset_size=10,
                                           prior_records=get_prior_records(failed_records),
                                           failed_records=failed_records)

### Re-run the above command whenever you want to add more new data to Elasticsearch. Remember to be nice about it! Don't do it in the middle of a workday, ask for 10,000 records at once, etc. Or don't do it at all because of its questionable legality. You can order the entire DB for $50 from their office.

# Step 2: inspect your data in Kibana

In [ ]:
# Kibana gives you those pretty graphs in iframes. They're nice and easy to put on a web page.
# I'll add them if I ever make a blog post about this.

## It's strange that there are records with InactiveDates from 1820 in my data. 
### Visual inspection of the data in Kibana shows that data with an InactiveDate is incredibly sparse until around 1985. I'm going to assume that Rhode Island didn't worry about putting in many inactive corporations into their corporate database when they created it. 
### It seems reasonable to exclude companies founded before 1984. However, that also puts a ceiling on the life expectancies of companies in my data. Whether they should be excluded or not depends on how you want to use this analysis. I'm going to leave them for now and view this as a study of companies that existed between 1985 and today, regardless of when they were founded.

# Step 3: load your data into pandas and inspect it more carefully

In [29]:
# Let's load the database into pandas. Our actual analysis later will only use samples that
# have an InactiveDate, so only request data with that data. We'd also want to filter out newer
# companies to only work with companies that have had a long time to go out of business, but
# reviewing our data in Kibana shows that our newest data is in 2007, implying that newer
# companies have a different URL pattern than older companies. No need to filter based on date organized.

data_to_load = elasticsearch.helpers.scan(
                    es,
                    query={"query": {"exists": {"field": "InactiveDate"}}},
                    index="ri_sos",
                    doc_type="corporate_data")

# Initialize a double ended queue
output_all = collections.deque()
# Extend deque with iterator
output_all.extend(data_to_load)
only_data = [item["_source"] for item in output_all]

df = json_normalize(only_data)
print(df.shape)

(32480, 23)


### Let's start by seeing what types of companies we have lots of data on.
### First get the most commmon words in the names of companies.

In [33]:
# Remove all punctuation from the names of companies
list_of_names = [row[1] for row in df['EntityName'].iteritems()]

list_of_names_no_punctuation = []
for name in list_of_names:
    list_of_names_no_punctuation += [re.sub('\,|\.|\!|\;|\:|\?|', '', name)]

# Break each name into a list of words so we can count the occurrence of each word
new_list = []
for words in list_of_names_no_punctuation:
    for word in words.split():
        new_list += [word.lower()]

# Count the occurrence of each word:
name_count = Counter(new_list)
popular_names = name_count.most_common(500)

# Print the 10 most popular names:
print(popular_names[:20])

[('inc', 18764), ('llc', 6263), ('&', 2499), ('corporation', 1647), ('of', 1365), ('the', 1315), ('company', 1293), ('co', 1147), ('realty', 1145), ('services', 1143), ('ltd', 1031), ('associates', 1019), ('corp', 962), ('island', 842), ('and', 796), ('rhode', 718), ('construction', 696), ('enterprises', 667), ('group', 647), ('incorporated', 618)]


In [34]:
# Let's make a 2nd list to see if the popularity changes much if we account for common word endings:
list_of_names_no_endings = []
for word in new_list:
    if word [-1:] == 's':
        list_of_names_no_endings += [word[:-1]]
    elif word[-3:] == 'ing':
        list_of_names_no_endings += [word[:-3]]
    else:
        list_of_names_no_endings += [word]

# Count the occurrence of each word:
name_count2 = Counter(list_of_names_no_endings)
popular_names2 = name_count2.most_common(500)

# Print the 10 most popular names:
print(popular_names2[:20])

[('inc', 18764), ('llc', 6263), ('&', 2499), ('corporation', 1648), ('service', 1638), ('of', 1365), ('the', 1315), ('company', 1293), ('co', 1148), ('realty', 1145), ('ltd', 1031), ('associate', 1021), ('corp', 969), ('island', 842), ('and', 796), ('rhode', 722), ('enterprise', 715), ('construction', 697), ('group', 648), ('incorporated', 618)]


### After some inspection, there wasn't much difference between the two lists for names of data. At least we tried. Let's filter out some unhelpful words and move on.

In [37]:
# There are a lot of unhelpful words in this list. Let's filter out some of the common ones:
boring_names = ['inc', 'llc', '&', 'corporation', 'the', 'of', 'company', 'associate', 'co', 'ltd', 'corp', 'and', 'enterprise',
               'association', 'group', 'associates', 'enterprises', 'incorporated', 'a', 'limited', 'm', 'r', 'c', 'partnership',
               'club', 'in', '', 'd', 'for', 'b', 'lp', 'e', 'h', 'g', 'k', 'p', 'l', '-', 'an', 's', 'i', 'f', 'j']
popular_and_not_boring = [item for item in popular_names if item[0] not in boring_names]
print(popular_and_not_boring[:80])

[('realty', 1145), ('services', 1143), ('island', 842), ('rhode', 718), ('construction', 696), ('management', 532), ('service', 495), ('new', 469), ('properties', 423), ('development', 405), ('international', 389), ('mortgage', 353), ('systems', 353), ('england', 333), ('ri', 332), ('center', 316), ('auto', 306), ('financial', 305), ('american', 291), ('providence', 264), ('street', 264), ('home', 246), ('insurance', 233), ('investments', 229), ('communications', 210), ('entity', 207), ('business', 207), ('solutions', 205), ('not', 203), ('qualified', 200), ('into', 197), ('consulting', 195), ('capital', 195), ('sales', 190), ('**merged', 189), ('builders', 189), ('marketing', 186), ('east', 185), ('agency', 182), ('holdings', 180), ('restaurant', 180), ('products', 178), ('real', 177), ('design', 176), ('newport', 176), ('investment', 174), ('estate', 173), ('foundation', 173), ('industries', 171), ('family', 162), ('building', 157), ('health', 153), ('state', 153), ('t', 152), ('nort

# Lifespan of businesses: a quick inspection
### Our goal is to see how long businesses in different industries stay in business. We don't have a variable for length in business yet, so let's make it.

In [38]:
# How many companies have out-of-business dates?
df = df.assign(time_in_business=pd.Series(np.nan))

# Convert date columns to the date format
df['InactiveDate'] = pd.to_datetime(df['InactiveDate'])
df['OrganisationDate'] = pd.to_datetime(df['OrganisationDate'])

# Add the variable 'time_in_business' when we can calculate it:
df.loc[df['InactiveDate'] != np.nan, 'time_in_business'] = (df['InactiveDate'] - df['OrganisationDate']).dt.days

# We're trying to predict time_in_business, so let's remove all rows without a value for that
df = df[df.time_in_business.notnull()]
print("Returned shape:", df.shape)

print(df.time_in_business.describe())

OverflowError: Overflow in int64 addition

### How to deal with negative values for time_in_business if you have them in your data

In [ ]:
# How is there a negative value for time_in_business? Your sample may not have one, but mine did.

#weird_row = df.loc[df.time_in_business == df.time_in_business.min()]
#print(weird_row.IDNumber)

In [ ]:
# On manual inspection, Rhode Island's records do in fact show at least one company as having been 
# dissolved before it was created. That's very strange. 
# Manually inspect & delete any records with negative lifespans.

#weird_rows = df.loc[df.time_in_business < 0]
#print(weird_rows.IDNumber)

In [ ]:
### I'm dropping the row that appeared in my data. Your data will most likely have a different row to drop.
# Don't run this cell without inspecting your own data!

#df.drop([weird_rows.index[0]], axis=0, inplace=True)

In [ ]:
# The max time_in_business number I have is 61626 days, or 169 years! Does Rhode Island really have that much data?
too_long = df.loc[df.time_in_business > 36500]
print(too_long[["IDNumber", "time_in_business"]])

# I manually inspected a few of these in my data, and they were legit. An old bank, an old cemetery, etc.
# There's a good argument to be made for excluding all businesses founded so long ago, but we're not going to deal with that now.

In [ ]:
# Let's graph the distribution to see what it looks like
plt.hist(df.time_in_business, bins = 365,
         color = 'blue', edgecolor = 'black')
plt.axis([0, 10000, 0, 700])
plt.title('Binwidth = 1 year', size = 30)
plt.xlabel('Days in business', size = 22)
plt.ylabel('# of companies', size= 22)

plt.tight_layout()
plt.show()

In [ ]:
# This variable is much more appropriate to work with as a log
first_tib = df[["time_in_business", "IDNumber"]].copy()
first_tib["log_tib"] = np.log(first_tib.time_in_business)
print(first_tib.log_tib.describe())

# Graph it:
plt.hist(first_tib.log_tib, bins = 35,
         color = 'blue', edgecolor = 'black')
plt.axis([0, 20, 0, 1200])
plt.xlabel('Log days in business', size = 22)
plt.ylabel('# of companies', size= 22)

plt.tight_layout()
plt.show()

In [ ]:
# This looks much nicer. Let's add it to the dataframe.
df["log_time_in_business"] = first_tib["log_tib"]

### We should inspect our other variables further, especially since we found some weird values in time_in_business. It's also worth inspecting whether we should log other variables, like the number of directors & officers. However, that's another bit of work I'm going to put off.

### Time to learn something. Which companies stay in business longer (ignoring the problem of our censored data)?

In [ ]:
# You can't dynamically create keywords in Python, so here's the long & ugly way to do this:
df = df.assign(contains_realty=pd.Series(False))
df = df.assign(contains_construction=pd.Series(False))
df = df.assign(contains_investment=pd.Series(False))
df = df.assign(contains_auto=pd.Series(False))
df = df.assign(contains_cleaning=pd.Series(False))
df = df.assign(contains_national=pd.Series(False))
df = df.assign(contains_state=pd.Series(False))
df = df.assign(contains_electric=pd.Series(False))
df = df.assign(contains_health=pd.Series(False))
df = df.assign(contains_marine=pd.Series(False))
df = df.assign(contains_marketing=pd.Series(False))
df = df.assign(contains_restaurant=pd.Series(False))
df = df.assign(contains_jewelry=pd.Series(False))
df = df.assign(contains_technologies=pd.Series(False))
df = df.assign(contains_supply=pd.Series(False))
df = df.assign(contains_church=pd.Series(False))
df = df.assign(contains_not=pd.Series(False))
df = df.assign(contains_new=pd.Series(False))

df.contains_realty = df.EntityName.str.contains('realty', case=False, regex=False)
df.contains_construction = df.EntityName.str.contains('construction', case=False, regex=False)
df.contains_investment = df.EntityName.str.contains('investment', case=False, regex=False)
df.contains_auto = df.EntityName.str.contains('auto', case=False, regex=False)
df.contains_cleaning = df.EntityName.str.contains('cleaning', case=False, regex=False)
df.contains_national = df.EntityName.str.contains('national', case=False, regex=False)
df.contains_state = df.EntityName.str.contains('state', case=False, regex=False)
df.contains_electric = df.EntityName.str.contains('electric', case=False, regex=False)
df.contains_health = df.EntityName.str.contains('health', case=False, regex=False)
df.contains_marine = df.EntityName.str.contains('marine', case=False, regex=False)
df.contains_marketing = df.EntityName.str.contains('marketing', case=False, regex=False)
df.contains_restaurant = df.EntityName.str.contains('restaurant', case=False, regex=False)
df.contains_jewelry = df.EntityName.str.contains('jewelry', case=False, regex=False)
df.contains_technologies = df.EntityName.str.contains('technologies', case=False, regex=False)
df.contains_supply = df.EntityName.str.contains('supply', case=False, regex=False)
df.contains_church = df.EntityName.str.contains('church', case=False, regex=False)
df.contains_not = df.EntityName.str.contains('not', case=False, regex=False)
df.contains_new = df.EntityName.str.contains('new', case=False, regex=False)

In [ ]:
# Average days in business for different types of businesses (only including businesses that have gone out of business)
print("Average life expectancy")
print("Realty:", df.loc[df.contains_realty == True, "time_in_business"].mean().days, "days")
print("Construction:", df.loc[df.contains_construction == True, "time_in_business"].mean().days, "days")
print("Investment:", df.loc[df.contains_investment == True, "time_in_business"].mean().days, "days")
print("Auto:", df.loc[df.contains_auto == True, "time_in_business"].mean().days, "days")
print("Cleaning:", df.loc[df.contains_cleaning == True, "time_in_business"].mean().days, "days")
print("National:", df.loc[df.contains_national == True, "time_in_business"].mean().days, "days")
print("State:", df.loc[df.contains_state == True, "time_in_business"].mean().days, "days")
print("Electric:", df.loc[df.contains_electric == True, "time_in_business"].mean().days, "days")
print("Health:", df.loc[df.contains_health == True, "time_in_business"].mean().days, "days")
print("Marine:", df.loc[df.contains_marine == True, "time_in_business"].mean().days, "days")
print("Marketing:", df.loc[df.contains_marketing == True, "time_in_business"].mean().days, "days")
print("Restaurant:", df.loc[df.contains_restaurant == True, "time_in_business"].mean().days, "days")
print("Jewelry:", df.loc[df.contains_jewelry == True, "time_in_business"].mean().days, "days")
print("Technologies:", df.loc[df.contains_technologies == True, "time_in_business"].mean().days, "days")
print("Supply:", df.loc[df.contains_supply == True, "time_in_business"].mean().days, "days")
print("Church:", df.loc[df.contains_church == True, "time_in_business"].mean().days, "days")
print("")
print("Just for curiosity:")
print("Not:", df.loc[df.contains_not == True, "time_in_business"].mean().days, "days")
print("New:", df.loc[df.contains_new == True, "time_in_business"].mean().days, "days")

### With the random sample I used, some differences were substantial. It looks like we may fhave some good predictors.
### Let's make the last few variables we're going to use: the number of directors & officers listed, a word-count for the Comments variable similar to what we did for company names, and dummy variables for the type of legal entity registered

In [ ]:
# The D&O data I grabbed isn't very good. I should have saved a count for each row containing data 
# in the Directors & Officers table, but I grabbed the values instead. 
# There are lots of clever things we could do, like count the number of words coercible into integers 
# (1 per street address plus 1 per zip code), count how often "USA" shows up as the country code, etc, but
# since this isn't my real, final product, I'm going to take the easy way out & count the number of characters
# present in the field. That should serve as a fine proxy.
df['officers_listed'] = df.apply(lambda row: len(row.d_and_o), axis=1)

In [ ]:
# The Comments variable is also called "Purpose" on Rhode Island's website. It's intended to be where companies describe
# what kind of business they intend to do. Most companies give at least a brief answer, but plenty don't.
# My favorite common fake response is "Any Lawful Purpose."

# Remove all punctuation and newline characters
list_of_comments = [row[1] for row in df.itertuples()]
list_of_comments_no_punctuation = []
for name in list_of_comments:
    name = name.replace('\n', ' ')
    list_of_comments_no_punctuation += [re.sub('\,|\.|\!|\;|\:|\?|', '', name)]

# Break each name into a list of words so we can count the occurrence of each word
new_list = []
for words in list_of_comments_no_punctuation:
    for word in words.split():
        new_list += [word.lower()]

# Count the occurrence of each word:
name_count = Counter(new_list)
popular_names = name_count.most_common(500)

In [ ]:
# Filter out the common & unhelpful words. Look at the resulting list to pick out worthwhile words to focus on.
# The number clusters are references to the Rhode Island code, typically Title 7.1, which has been repealed.
# There's definitely some good information in which code people are quoting, but it'd take more research to discover that.
boring_names = ['inc', 'llc', '&', 'corporation', 'the', 'of', 'company', 'associate', 'co', 'ltd', 'corp', 'and', 'enterprise',
               'association', 'group', 'associates', 'enterprises', 'incorporated', 'a', 'limited', 'm', 'r', 'c', 'partnership',
               'club', 'in', '', 'd', 'for', 'b', 'lp', 'e', 'h', 'g', 'k', 'p', 'l', '-', 'an', 's', 'title', '7-6', '7-16',
               '7-11-51', 'to', '7-11', '7-12-1405', '7-13-8', '7', '7-12-1701', 'as', '116', 'on', 'gl', 'at', '1956',
               'business', 'operate']
popular_and_not_boring = [item for item in popular_names if item[0] not in boring_names]
#print(popular_and_not_boring[:80])

In [ ]:
# You can't dynamically create keywords in Python, so here's the long & ugly way to do this:
df = df.assign(comments_contains_retail=pd.Series(False))
df = df.assign(comments_contains_management=pd.Series(False))
df = df.assign(comments_contains_construction=pd.Series(False))
df = df.assign(comments_contains_property=pd.Series(False))
df = df.assign(comments_contains_commercial=pd.Series(False))
df = df.assign(comments_contains_residential=pd.Series(False))
df = df.assign(comments_contains_mortgage=pd.Series(False))
df = df.assign(comments_contains_rental=pd.Series(False))
df = df.assign(comments_contains_development=pd.Series(False))
df = df.assign(comments_contains_restaurant=pd.Series(False))
df = df.assign(comments_contains_manage=pd.Series(False))
df = df.assign(comments_contains_purchase=pd.Series(False))
df = df.assign(comments_contains_food=pd.Series(False))
df = df.assign(comments_contains_insurance=pd.Series(False))
df = df.assign(comments_contains_equipment=pd.Series(False))
df = df.assign(comments_contains_holding=pd.Series(False))
df = df.assign(comments_contains_investment=pd.Series(False))
df = df.assign(comments_contains_manufacture=pd.Series(False))
df = df.assign(comments_contains_design=pd.Series(False))
df = df.assign(comments_contains_distribution=pd.Series(False))
df = df.assign(comments_contains_maintenance=pd.Series(False))
df = df.assign(comments_contains_practice=pd.Series(False))
df = df.assign(comments_contains_computer=pd.Series(False))
df = df.assign(comments_contains_contracting=pd.Series(False))
df = df.assign(comments_contains_health=pd.Series(False))
df = df.assign(comments_contains_medical=pd.Series(False))
df = df.assign(comments_contains_marketing=pd.Series(False))
df = df.assign(comments_contains_installation=pd.Series(False))
df = df.assign(comments_contains_jewelry=pd.Series(False))
df = df.assign(comments_contains_educational=pd.Series(False))
df = df.assign(comments_contains_engineering=pd.Series(False))
df = df.assign(comments_contains_community=pd.Series(False))

df.comments_contains_retail = df.Comments.str.contains('retail', case=False, regex=False)
df.comments_contains_management = df.Comments.str.contains('management', case=False, regex=False)
df.comments_contains_construction = df.Comments.str.contains('construction', case=False, regex=False)
df.comments_contains_property = df.Comments.str.contains('property', case=False, regex=False)
df.comments_contains_development = df.Comments.str.contains('development', case=False, regex=False)
df.comments_contains_commercial = df.Comments.str.contains('commercial', case=False, regex=False)
df.comments_contains_residential = df.Comments.str.contains('residential', case=False, regex=False)
df.comments_contains_mortgage = df.Comments.str.contains('mortgage', case=False, regex=False)
df.comments_contains_rental = df.Comments.str.contains('rental', case=False, regex=False)
df.comments_contains_restaurant = df.Comments.str.contains('restaurant', case=False, regex=False)
df.comments_contains_manage = df.Comments.str.contains('manage', case=False, regex=False)
df.comments_contains_purchase = df.Comments.str.contains('purchase', case=False, regex=False)
df.comments_contains_food = df.Comments.str.contains('food', case=False, regex=False)
df.comments_contains_insurance = df.Comments.str.contains('insurance', case=False, regex=False)
df.comments_contains_equipment = df.Comments.str.contains('equipment', case=False, regex=False)
df.comments_contains_holding = df.Comments.str.contains('holding', case=False, regex=False)
df.comments_contains_investment = df.Comments.str.contains('investment', case=False, regex=False)
df.comments_contains_manufacture = df.Comments.str.contains('manufacture', case=False, regex=False)
df.comments_contains_design = df.Comments.str.contains('design', case=False, regex=False)
df.comments_contains_distribution = df.Comments.str.contains('distribution', case=False, regex=False)
df.comments_contains_maintenance = df.Comments.str.contains('maintenance', case=False, regex=False)
df.comments_contains_practice = df.Comments.str.contains('practice', case=False, regex=False)
df.comments_contains_computer = df.Comments.str.contains('computer', case=False, regex=False)
df.comments_contains_contracting = df.Comments.str.contains('contracting', case=False, regex=False)
df.comments_contains_health = df.Comments.str.contains('health', case=False, regex=False)
df.comments_contains_medical = df.Comments.str.contains('medical', case=False, regex=False)
df.comments_contains_marketing = df.Comments.str.contains('marketing', case=False, regex=False)
df.comments_contains_installation = df.Comments.str.contains('installation', case=False, regex=False)
df.comments_contains_jewelry = df.Comments.str.contains('jewelry', case=False, regex=False)
df.comments_contains_educational = df.Comments.str.contains('educational', case=False, regex=False)
df.comments_contains_engineering = df.Comments.str.contains('engineering', case=False, regex=False)
df.comments_contains_community = df.Comments.str.contains('community', case=False, regex=False)

In [ ]:
# Make dummy variables for all the types of legal entities that have been registered. We'll use all of them
# but one in the analysis ('Foreign Corporation').
reg_df = pd.concat([df, pd.get_dummies(df['EntityType'])], axis=1)

In [ ]:
explanatory_variables = ['comments_contains_retail', 'comments_contains_management', 'comments_contains_construction',
                        'comments_contains_property', 'comments_contains_development', 'comments_contains_commercial',
                        'comments_contains_residential', 'comments_contains_mortgage', 'comments_contains_rental',
                        'comments_contains_restaurant', 'comments_contains_manage', 'comments_contains_purchase',
                        'comments_contains_food', 'comments_contains_insurance', 'comments_contains_equipment',
                        'comments_contains_holding', 'comments_contains_investment', 'comments_contains_manufacture',
                        'comments_contains_design', 'comments_contains_distribution', 'comments_contains_maintenance',
                        'comments_contains_practice', 'comments_contains_computer', 'comments_contains_contracting',
                        'comments_contains_health', 'comments_contains_medical', 'comments_contains_marketing',
                        'comments_contains_installation', 'comments_contains_jewelry', 'comments_contains_educational',
                        'comments_contains_engineering', 'comments_contains_community',
                        'contains_realty', 'contains_construction', 'contains_investment', 'contains_auto', 
                        'contains_cleaning', 'contains_national', 'contains_state', 'contains_electric', 'contains_health',
                        'contains_marine', 'contains_marketing', 'contains_restaurant', 'contains_jewelry',
                        'contains_technologies', 'contains_supply', 'contains_church',
                        'Bank', 'Credit Union', 'Domestic Limited Liability Company', 'Domestic Limited Liability Partnership',
                        'Domestic Limited Partnership', 'Domestic Non-Profit Corporation', 'Domestic Profit Corporation',
                        'Foreign Limited Liability Company', 'Foreign Limited Partnership', 'Foreign Non-Profit Corporation',
                        'Foreign Registered Limited Liability Partnership', 'Insurance', 'Professional Service Corporation',
                        'officers_listed']

# time_in_business as days
X_train, X_test, y_train, y_test = train_test_split(reg_df[explanatory_variables],
                                                    reg_df['time_in_business'],
                                                    test_size=0.2,
                                                    random_state=5)

# log_time_in_business
X_train_logged, X_test_logged, y_train_logged, y_test_logged = train_test_split(reg_df[explanatory_variables],
                                                                reg_df['log_time_in_business'],
                                                                test_size=0.2,
                                                                random_state=5)

# Step 5: make some models

In [ ]:
# Bigger question: if I give you details about a new company, can you predict how long they'll be in business?
# That's appropriate for a normal linear regression, assuming we ignore the censoring issue for still-in-business companies.

## Start with a basic elastic net model, modeling time in business in days (not the log)

In [ ]:
# We have so many variables here that overfitting is a reasonable concern. Let's go with an elastic net model
# as a first line of defense.

e_net = sm.OLS(y_train, X_train.astype(float), L1_wt=.5)
print(e_net.fit().summary())

In [ ]:
# This model is clearly overspecified. Clear out the worst performing variables & see what happens
vars_to_keep = []
for item in explanatory_variables:
    if e_net.fit().pvalues[item] < 0.1:
        vars_to_keep += [item]

smaller_X_train = X_train[vars_to_keep]

e_net2 = sm.OLS(y_train, smaller_X_train.astype(float), L1_wt=.5)
print(e_net2.fit().summary())

## An adjusted R^2 of 44% is crazy. How can we know so much about the life expectancy of a company from such a small amount of information?

In [ ]:
#test_fit = sm.OLS(y_test, X_test[vars_to_keep].astype(float), L1_wt=.5)

#e_fit2 = test_fit.fit()
#test_fit = e_fit2.predict(X_test[vars_to_keep])

# Set an index for both series
#test_fit.reset_index(drop=True, inplace=True)
#y_test_indexed = y_test.reset_index(drop=True)

# Put them in the same DF
#comparison_df = pd.concat([test_fit, y_test_indexed], axis=1)
#comparison_df.sort_values("time_in_business", inplace=True)

In [ ]:
#### Graph predictions vs actuals. Order by size of the actual



## Now use the same elastic net model with logged days_in_business

In [ ]:
e_net_logged = sm.OLS(y_train_logged, X_train_logged.astype(float), L1_wt=.5)
print(e_net_logged.fit().summary())

In [ ]:
# Once more, dropping a bunch of poor predictors:
vars_to_keep = []
for item in explanatory_variables:
    if e_net_logged.fit().pvalues[item] < 0.1:
        vars_to_keep += [item]

smaller_X_train_logged = X_train_logged[vars_to_keep]

e_net2_logged = sm.OLS(y_train_logged, smaller_X_train_logged.astype(float), L1_wt=.5)
print(e_net2_logged.fit().summary())

## Our adjusted R^2 is now 87%. We've done no model tuning whatsoever. I'm astonished.
### Also notice that we had 45 worthwhile variables in this last model, compared to 27 in the non-logged model.

In [ ]:
# Graph the results from test values against their actual values.

#test_fit = sm.OLS(y_test, X_test[vars_to_keep].astype(float), L1_wt=.5)
test_fit_logged = sm.OLS(y_test_logged, X_test_logged[vars_to_keep].astype(float), L1_wt=.5)
#e_fit2_logged = e_net2_logged.fit()

e_fit2_logged = test_fit_logged.fit()
test_fit_logged2 = e_fit2.predict(X_test_logged[vars_to_keep])

# Set an index for both series
test_fit_logged2.reset_index(drop=True, inplace=True)
y_test_indexed_logged = y_test.reset_index(drop=True)

# Put them in the same DF
comparison_df_logged = pd.concat([test_fit_logged2, y_test_indexed_logged], axis=1)
comparison_df_logged.sort_values("time_in_business", inplace=True)





#### Graph predictions vs actuals. Order by size of the actual

#fig, ax = plt.subplots()
#ax.plot(x1, y, 'o', label="Data")
#ax.plot(x1, y_true, 'b-', label="True")
#ax.plot(np.hstack((x1, x1n)), np.hstack((ypred, ynewpred)), 'r', label="OLS prediction")
#ax.legend(loc="best");

#fig, ax = plt.subplots()
#fig = sm.graphics.plot_fit(test_fit, y_test_indexed, ax=ax)
#ax.set_ylabel("Murder Rate")
#ax.set_xlabel("Poverty Level")
#ax.set_title("Linear Regression")

plt.plot(comparison_df_logged[0], comparison_df_logged["time_in_business"], 'ro')
plt.axis([0, 15000, 0, 15000])

plt.show()



In [ ]:
# Finish by comparing our linear regression model to Random Forests, where I include the same variables,
# then another where I throw in more (all) variables.

In [ ]:
# rf_df = 

# This is far from a completed study. 

### Things left to do:
    - There's a colinearity problem introduced by having separate variables for keywords in both EntityName and Comments: lots of companies with names like ABC Insurance will also say in their description that they sell insurance. Instead, the keyword variables should apply to both EntityName and Comments.
    - Similarly, I should figure out some clever way to make different variables for all words in EntityName & Comments that's present more than X amount of times. This whole inspect-by-hand thing isn't scalable and leaves too much room for my own biases in what names are important.
    - Assess the impact of our data being censored (companies that are still in business aren't included in the analysis, which biases our results towards earlier deaths)
    - Not all of these businesses failed. Some were sold, merged, etc. An example is this: http://ucc.state.ri.us/CorpWeb/CorpSearch/CorpSummary.aspx?FEIN=000001992. The issue is figuring out the structure of the pages scraped to know how to scrape it.
    - Some of the data we used is subject to change over a company's lifetime. Addresses, DBAs, or the number of directors & officers can all change. Rhode Island includes lots of other documents, and it'd be great to see what other data we can get from those.
    - Overlay census data based on companies' principal addresses to see how population density or nearby wealth impact their life expectancy
    - Use our other variables like stock information, NAICS codes, or which foreign entities use registered agents
    - Consider macroeconomic factors: GDP growth, unemployment rates, etc
    - Test for unit roots: are there trends over time that we need to account for? Are survival rates different for companies founded in 1900 vs 1980? Do survival rates for different industruies, like technology or construction companies, change significantly over time due to structural changes (not due to macroeconomic factors that we can control for)?
    - More data is always better. I only used 10,000 samples in this study.
### An extension of this study:
    - It's neat to be able to estimate a company's lifespan at birth. It'd be useful to estimate how much longer a company will stay in business, regardless of how old it already is.

In [ ]:
# If you've done much scraping, then your failed_records list could be pretty long. You don't want to
# repeat attempting to scrape those failed records, so save the list to a file.

#with open('failed_records', 'wb') as fp:
#    pickle.dump(failed_records, fp)